In [4]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import matplotlib.pyplot as plt
import numpy as np

In [5]:
cuda = torch.cuda.is_available()
batch_size = 128
log_interval = 10
num_epochs = 15

torch.manual_seed(1) # args.seed

device = torch.device("cuda" if cuda else "cpu") # args.cuda
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {} # args.cuda

# Get train and test data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)



9913344it [00:00, 33378216.75it/s]                             


Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw



29696it [00:00, 29648667.36it/s]         

Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw




1649664it [00:00, 16015796.18it/s]                             


Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw



5120it [00:00, 5143673.41it/s]          

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw



In [6]:
# original VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc1a = nn.Linear(400, 100)
        self.fc21 = nn.Linear(100, 2) # Latent space of 2D
        self.fc22 = nn.Linear(100, 2) # Latent space of 2D
        self.fc3 = nn.Linear(2, 100) # Latent space of 2D
        self.fc3a = nn.Linear(100, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc1a(h1))
        return self.fc21(h2), self.fc22(h2)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.relu(self.fc3a(h3))
        return torch.sigmoid(self.fc4(h4))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD # -ELBO


def train(epoch):
    model.train() # so that everything has gradients and we can do backprop and so on...
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad() # "reset" gradients to 0 for text iteration
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward() # calc gradients
        train_loss += loss.item()
        optimizer.step() # backpropagation

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad(): # no_grad turns of gradients...
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


for epoch in range(1, num_epochs + 1):
    train(epoch)
    torch.save(model.state_dict(), "weights/VAE.pth")
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 2).to(device) # 20 -> 2
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/VAE/sample_' + str(epoch) + '.png')

====> Epoch: 1 Average loss: 187.0061
====> Test set loss: 166.7224
====> Epoch: 2 Average loss: 162.5338
====> Test set loss: 159.1779
====> Epoch: 3 Average loss: 157.2969
====> Test set loss: 155.1728
====> Epoch: 4 Average loss: 153.7888
====> Test set loss: 152.7887
====> Epoch: 5 Average loss: 151.5472
====> Test set loss: 151.0308
====> Epoch: 6 Average loss: 149.6894
====> Test set loss: 149.1899
====> Epoch: 7 Average loss: 148.4808
====> Test set loss: 148.7808
====> Epoch: 8 Average loss: 147.4285
====> Test set loss: 147.6596
====> Epoch: 9 Average loss: 146.6077
====> Test set loss: 146.9002
====> Epoch: 10 Average loss: 145.9734
====> Test set loss: 146.6191
====> Epoch: 11 Average loss: 145.2647
====> Test set loss: 145.7903
====> Epoch: 12 Average loss: 144.5741
====> Test set loss: 145.3732
====> Epoch: 13 Average loss: 144.3107
====> Test set loss: 145.4503
====> Epoch: 14 Average loss: 143.6761
====> Test set loss: 145.3122
====> Epoch: 15 Average loss: 143.4321
====

In [7]:
class convVAE(nn.Module):
    def __init__(self, channels=1, features=32*20*20, hiddenDim=256):
        super(convVAE, self).__init__()

        self.conv1 = nn.Conv2d(channels, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 5)
        
        self.enFc1 = nn.Linear(features, hiddenDim)
        self.enFc2 = nn.Linear(features, hiddenDim)

        self.deFc1 = nn.Linear(hiddenDim, features)
        self.deConv1 = nn.ConvTranspose2d(32, 16, 5)
        self.deConv2 = nn.ConvTranspose2d(16, channels, 5)


    def encode(self, x):
        h1 = F.relu(self.conv1(x))
        h2 = F.relu(self.conv2(h1))
        h3 = h2.view(-1, 32*20*20)
        return self.enFc1(h3), self.enFc2(h3)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h4 = F.relu(self.deFc1(z))
        h5 = F.relu(self.deConv1(h4.view(-1, 32, 20, 20)))
        return F.sigmoid(self.deConv2(h5))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = convVAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD # -ELBO



In [12]:
for epoch in range(1, num_epochs + 1):
    train(epoch)
    torch.save(model.state_dict(), "weights/convVAE.pth")
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, batch_size*2).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/convVAE/sample_' + str(epoch) + '.png')

c:\Users\kjart\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
c:\Users\kjart\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


====> Epoch: 1 Average loss: 110.8238
====> Test set loss: 108.8716
====> Epoch: 2 Average loss: 108.6781
====> Test set loss: 107.1070
====> Epoch: 3 Average loss: 107.2825
====> Test set loss: 105.9744
====> Epoch: 4 Average loss: 106.3306
====> Test set loss: 105.1743
====> Epoch: 5 Average loss: 105.5387
====> Test set loss: 104.7510
====> Epoch: 6 Average loss: 105.0378
====> Test set loss: 104.0637
====> Epoch: 7 Average loss: 104.4307
====> Test set loss: 103.7819
====> Epoch: 8 Average loss: 104.0330
====> Test set loss: 103.3447
====> Epoch: 9 Average loss: 103.7696
====> Test set loss: 102.8881
====> Epoch: 10 Average loss: 103.2829
====> Test set loss: 102.5924
====> Epoch: 11 Average loss: 103.0197
====> Test set loss: 102.5310
====> Epoch: 12 Average loss: 102.7559
====> Test set loss: 102.0627
====> Epoch: 13 Average loss: 102.4448
====> Test set loss: 102.0999
====> Epoch: 14 Average loss: 102.2870
====> Test set loss: 101.6893
====> Epoch: 15 Average loss: 102.0573
====

In [ ]:
# https://colab.research.google.com/drive/1sjy9odlSSy0RBVgMTgP7s99NXsqglsUL?usp=sharing#scrollTo=Rj17psVw7Shg
# https://www.youtube.com/watch?v=a4Yfz2FxXiY&t=3s 

In [10]:
64*64

4096